In [ ]:
%matplotlib widget

## Exercise No. 6 -- Outgoing Longwave Radiation (OLR)

#### 1)

Run ARTS on the Jupyter notebook olr.ipynb . This will calculate the spectrum
of outgoing longwave radiation for a midlatitude-summer atmosphere.
Our calculation trades accuracy for computational efficiency. For
example, we use only water vapor and carbon dioxide as absorbers.
We use only 300 frequency grid points and approximately 54,000 spectral
lines, whereas for accurate calculations one needs at least 10,000
frequency grid points and 500,000 spectral lines, taking into account
absorbing species like ozone and methane. The script plots the spectral
irradiance, in SI units, as a function of wavenumber. Planck curves
for different temperatures are shown for comparison. We integrate
the whole spectrum to quantify the power per square that is emitted
by the atmosphere (value in title).

* How would the OLR spectrum look in units of brightness temperature?
* How would the Planck curves look in units of brightness temperature?
* Find the $\text{CO}_{2}$ absorption band and the regions of $\text{H}_{2}\text{O}$ 
absorption. From which height in the atmosphere does the radiation in the $\text{CO}_{2}$ band originate?
* Are there window regions?
* What will determine the OLR in the window regions?
* Use the plot to explain the atmospheric greenhouse effect.

In [ ]:
# Set path to line catalogue and xml-data
import os
# Uncomment and adjust the following lines
# if ARTS_DATA_PATH is not set as environment variable
# os.environ["ARTS_DATA_PATH"] = "your_path_to/arts-xml-data/:your_path_to/arts-cat-data/:"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pyarts import xml
from pyarts.arts import convert
from olr_module import calc_olr_from_atmfield, Change_T_with_RH_const, cmap2rgba, planck


In [ ]:
# Read input atmosphere
atmfield = xml.load("input/midlatitude-summer.xml")

# Scale the CO2 concentration
atmfield.scale("abs_species-CO2", 1)

# Add a constant value to the temperature
atmfield.set("T", atmfield.get("T") + 0)

# Add a constant value to the temperature but 
# without changing relative humidity 
atmfield = Change_T_with_RH_const(atmfield,DeltaT=0)

# Calculate the outgoing-longwave radiation
f, olr = calc_olr_from_atmfield(atmfield, verbosity=0)

In [ ]:
# Plotting.
import matplotlib
font = {'size'   : 24}
matplotlib.rc('font', **font)

wn = convert.freq2kaycm(f)

temps = [225, 250, 275, atmfield.get("T", keep_dims=False)[0]]
temp_colors = cmap2rgba("plasma", len(temps))

fig, ax = plt.subplots()
for t, color in sorted(zip(temps, temp_colors)):
    ax.plot(
        wn, np.pi * planck(f, t), label=f"{t:3.1f} K", color=color
    )
ax.plot(wn, olr, color="C0", label="Irradiance")
ax.legend()
ax.set_title(rf"OLR={np.trapz(olr, f):3.2f} $\sf Wm^{{-2}}$")
ax.set_xlim(wn.min(), wn.max())
ax.set_xlabel(r"Wavenumber [$\sf cm^{-1}$]")
ax.set_ylabel(r"Irradiance [$\sf Wm^{-2}Hz^{-1}$]")
ax.set_ylim(bottom=0)
fig.savefig("plots/olr.pdf")

#### 2) 

Investigate how the OLR changes for different atmospheric conditions
by modifying the input data.  
Use `atmfield.scale(...)`,
`atmfield.set(...)` and `Change_T_with_RH_const(...)` to change the atmospheric data:

* Add $1\,\,\text{K}$ to the temperature.
* Add $1\,\,\text{K}$ to the temperature but hold relative humidity constant.
* Increase the $\text{CO}_{2}$ conentration by a factor of $2$.
* Increase the $\text{H}_{2}\text{O}$ conentration by a factor of $1.2$.
   
1) Change it, and calculate and plot the spectrum for each change. 
2) Compare the spectra of the changed atmosphere with the unchanged OLR spectrum. Where
   do the changes occur? Explain the differnt effects of the changed atmospheres.
3) Compare the OLR numbers, which is the more potent greenhouse gas,
$\text{CO}_{2}$ or $\text{H}_{2}\text{O}$?